# ***<span style="color:green">Web Crawler</span>***

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md

# Tìm kiếm URL
# query = "Công ty Nexcel Solutions thành lập vào năm nào"
# search_results = list(search(query, num_results=6, advanced=True))
# if search_results:
#     url = search_results[0].url
#     print(f"Found URL: {url}")
# else:
#     print("No URL found.")
#     exit()
url = "https://nexcel.info/nexcel-career-development-handbook/"
# Lấy nội dung từ URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Lấy nội dung chính (giả sử trong thẻ <body>)
    content = soup.get_text(separator="\n")
    
    # Lưu vào file .txt
    with open("./employee_handbook.txt", "w", encoding="utf-8") as txt_file:
        txt_file.write(content)
    
    # Chuyển đổi sang Markdown
    markdown_content = md(html_content)
    with open("./employee_handbook.md", "w", encoding="utf-8") as md_file:
        md_file.write(markdown_content)
    
    print("Content saved to 'employee_handbook.txt' and 'employee_handbook.md'")
else:
    print(f"Failed to fetch the URL: {response.status_code}")


Content saved to 'employee_handbook.txt' and 'employee_handbook.md'


In [12]:
print(search_results)

[SearchResult(url=https://www.itjobs.com.vn/vi/employer/profile/nexcel-solutions-246, title=Nexcel Solutions - ITJobs.com.vn, description=Công TY TNHH Một Thành Viên Giải Pháp NEXCEL - Nexcel Solutions, một nhà ... ITJobs được thành lập vào năm 2014 tại Việt Nam và mục tiệu chính là trở ...), SearchResult(url=https://vn.linkedin.com/company/nexcel-solutions, title=Nexcel Solutions, description=Quy mô công ty: 201-500 nhân viên. Trụ sở: Ho Chi Minh, Ho Chi Minh. Loại hình: Privately Held. Thành lập: 2011. Vị trí. Chính. SMS Tower, Quang Trung Software ...), SearchResult(url=https://hosocongty.vn/cong-ty-tnhh-mot-thanh-vien-giai-phap-nexcel-com-83090.htm, title=công ty tnhh một thành viên giải pháp nexcel, description=Jan 31, 2023 — CÔNG TY TNHH MỘT THÀNH VIÊN GIẢI PHÁP NEXCEL · Đại diện pháp luật: AKAPAT RUKSAPHUN · Điện thoại: · Ngày cấp: 21/01/2010 · Ngành nghề chính: Tư vấn ...), SearchResult(url=https://www.topcv.vn/cong-ty/nexcel-solutions/116877.html, title=Nexcel Solutions, descr

In [77]:
import re

markdown_content = re.sub("\n\n+", "\n", markdown_content)

In [85]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(markdown_content)

In [ ]:
print(md_header_splits[4].page_content)

In [ ]:
from bs4 import BeautifulSoup
from markdown import markdown

html = markdown(md_header_splits[4].page_content)
text = ''.join(BeautifulSoup(html).findAll(text=True))

In [ ]:
print(text)

In [97]:
response = requests.get('http://nexcel.info/wp-content/uploads/2021/05/promotion-committee-assessment-form.jpg')

In [98]:
response

<Response [200]>

In [89]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)

In [96]:
print(splits[100].page_content)

| 3 | A | Assistant Manager | Senior Project Manager | 5+ years |
|  | B |  |  |  |
|  | C |  |  |  |
| 4 | A | Supervisor/Tech Supervisor | Project Manager/Assistant Project Manager | Project Manager: 3+ years Assistant Project Manager: <3 years |


In [68]:
from duckduckgo_search import DDGS
import re
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

def gg_search(query):
    # results = DDGS().text(query, max_results=5)
    # print(results)
    # urls = []
    # for result in results:
    #     url = result['href']
    #     urls.append(url)
    search_results = list(search(query, num_results=6, advanced=True))
    contents = []
    for result in search_results:
        response = requests.get(result.url)
        html_content = response.text
        soup = BeautifulSoup(html_content, "html.parser")
        content = soup.get_text(separator="\n")
        page_text = re.sub("\n\n+", "\n", content)
        # text = truncate(page_text)
        contents.append(page_text)

    return contents

def ddg_search(query):
    results = DDGS().text(query, max_results=5)
    print(results)
    urls = []
    for result in results:
        url = result['href']
        urls.append(url)

    docs = get_page(urls)

    content = []
    for doc in docs:
        page_text = re.sub("\n\n+", "\n", doc.page_content)
        # text = truncate(page_text)
        content.append(page_text)

    return content

def get_page(urls):
    loader = AsyncChromiumLoader(urls)
    html = loader.load()

    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["p"], remove_unwanted_tags=["a"])

    return docs_transformed

def truncate(text):
    words = text.split()
    truncated = " ".join(words[:400])

    return truncated 

In [69]:
content = gg_search("Công ty Nexcel Solutions thành lập vào năm nào ?")

In [43]:
path = 'https://nexcel.info/employee-handbook-2/'
response = requests.get(path)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
all_text = soup.get_text(separator="\n")


In [46]:
page_text = re.sub("\n\n+", "\n", all_text)

In [ ]:
t = re.sub("\n\n+", "\n", markdown)
print(t)

In [50]:
markdown = md(html_content)

c:\Users\leon.pham\AppData\Local\anaconda3\envs\langchain\Lib\site-packages\bs4\element.py:955: RuntimeWarning: coroutine 'AsyncChromiumLoader.ascrape_playwright' was never awaited
  u.setup()


In [53]:
print(type(markdown))

<class 'str'>


In [ ]:
print(page_text)

In [ ]:
print(content[0])

In [73]:
links = [a['href'] for a in soup.find_all('a', href=True)]

# ***<span style="color:green">RAG</span>***

In [ ]:
# To install all available features
!pip3 install "crawl4ai[all]"

# After installation, download the necessary models for better performance:
!crawl4ai-download-models

# Lastly, install Playwright dependencies
!playwright install 

In [2]:
with open('D:\Desktop\chatbot4group\data\Employee Handbook – Nexcel Info Site.html', 'r') as file:
    html_as_string = file.read()

In [ ]:
print(html_as_string)

In [1]:
import requests

url = 'https://nexcel.info/employee-handbook-2/'  # Replace with your desired URL
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    print('Failed to retrieve the document')

In [ ]:
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Pretty-printing the parsed HTML
print(soup.prettify())

In [4]:
title = soup.title.string
print(f'Page Title: {title}')

Page Title: Employee Handbook – Nexcel Info Site


In [ ]:
all_text = soup.get_text()
print(all_text)

In [ ]:
links = [a['href'] for a in soup.find_all('a', href=True)]
print('Links Found:', links)

In [ ]:
data = {
    'title': title,
    'text': all_text,
    'links': links
}

print('Structured Data:', data)

In [ ]:
from langchain.schema import Document

document = Document(
    page_content=data['text'],
    metadata={
        'title': data['title'],
        'links': data['links'],
    }
)

print('LangChain Document Created:', document)

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitting the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter()
text_splitter._chunk_size = 500
text_splitter._chunk_overlap = 20
documents = text_splitter.split_documents([document])

print(f'Number of Chunks Created: {len(documents)}')

Number of Chunks Created: 164


In [26]:
import re


def format_text(text):
    # Step 1: Remove excess whitespace and ensure consistent formatting
    text = re.sub(r'\n+', '\n', text.strip())
    
    # Step 2: Break the text into major sections based on patterns like '------'
    sections = re.split(r'\n[-=]+\n', text)
    
    formatted_text = ''
    
    for section in sections:
        # Step 3: For each section, process the title and content
        lines = section.strip().split('\n')
        title = lines[0]
        content = '\n'.join(lines[1:])
        
        # Step 4: Add title to formatted text
        formatted_text += title + '\n' + '-' * len(title) + '\n'
        
        # Step 5: Handle indentation for nested bullet points
        formatted_text += format_content(content) + '\n\n'
    
    return formatted_text.strip()



def format_content(content):
    lines = content.split('\n')
    formatted_lines = []
    indent_level = 0

    for line in lines:
        if line.startswith('-'):
            # Bullet points, so we add indentation
            formatted_lines.append('  ' * indent_level + line)
        else:
            # Non-bullet point, reset indentation
            if line.strip() == '':
                formatted_lines.append('')
            else:
                formatted_lines.append('  ' * indent_level + line)
        
        # Increase indent if the line is a section heading
        if re.search(r'[A-Za-z\s]+$', line):
            indent_level += 1
    
    return '\n'.join(formatted_lines)



def format_content_for_markdown(content):
    lines = content.split('\n')
    formatted_lines = []
    indent_level = 0

    for line in lines:
        if line.startswith('-'):
            # Bullet points, add indentation
            formatted_lines.append('  ' * indent_level + line)
        else:
            # Non-bullet point, reset indentation
            if line.strip() == '':
                formatted_lines.append('')
            else:
                formatted_lines.append('  ' * indent_level + line)
        
        # Increase indent if the line is a sub-section heading
        if re.search(r'[A-Za-z\s]+$', line):
            indent_level += 1
    
    return '\n'.join(formatted_lines)



def format_to_markdown(text):
    # Step 1: Remove excess whitespace and ensure consistent formatting
    text = re.sub(r'\n+', '\n', text.strip())
    
    # Step 2: Break the text into major sections based on patterns like '------'
    sections = re.split(r'\n[-=]+\n', text)
    
    markdown_text = ''
    
    for section in sections:
        # Step 3: For each section, process the title and content
        lines = section.strip().split('\n')
        title = lines[0]
        content = '\n'.join(lines[1:])
        
        # Step 4: Add title to markdown text as H1
        markdown_text += f'# {title}\n\n'
        
        # Step 5: Handle indentation for nested bullet points
        markdown_text += format_content_for_markdown(content) + '\n\n'
    
    return markdown_text.strip()

In [27]:
t = format_content(documents[0].page_content)

In [1]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
  "Write a delicious recipe for {dish} with a {flavor} twist."
)

# Formatting the prompt with new content
formatted_prompt = prompt_template.format(dish="pasta", flavor="spicy")

print(formatted_prompt)

Write a delicious recipe for pasta with a spicy twist.


In [ ]:
import time

def nextSquare():
    i = 1
 
    # An Infinite loop to generate squares
    while True:
        yield i*i
        i += 1  # Next execution resumes
        time.sleep(4)
        # from this point
 
 
# Driver code to test above generator
# function
for num in nextSquare():
    if num > 100:
        break
    print(num)

In [2]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "What is the tallest mountain in the world?",
        "answer": "Mount Everest",
    },
    {"question": "What is the largest ocean on Earth?", "answer": "Pacific Ocean"},
    {"question": "In which year did the first airplane fly?", "answer": "1903"},
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}",
)
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt_template.format(
        input="What is the name of the famous clock tower in London?"
    )
)

Question: What is the tallest mountain in the world?
Mount Everest

Question: What is the largest ocean on Earth?
Pacific Ocean

Question: In which year did the first airplane fly?
1903

Question: What is the name of the famous clock tower in London?
